In [5]:
import pandas as pd
import torch
import torch.nn as nn
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# 데이터 불러오기
df = sns.load_dataset('mpg')

# 결측치 제거
df.dropna(inplace=True)

In [8]:
# 입력 데이터와 타깃 데이터 분리하기
inputs = df.drop(['mpg', 'name'], axis=1)
inputs = pd.get_dummies(inputs)
outputs = df['mpg']

# 데이터를 훈련용과 검증용으로 나누기
X_train, X_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.2, random_state=42)

# 텐서로 변환하기
X_train_tensor = torch.tensor(X_train.values).float()
y_train_tensor = torch.tensor(y_train.values).float()
X_test_tensor = torch.tensor(X_test.values).float()
y_test_tensor = torch.tensor(y_test.values).float()

In [7]:
X_train_tensor.shape

torch.Size([313, 9])

* super(MultivariateLinearRegression, self).__init__()
    * 파이썬에서 다중 상속(multiple inheritance)을 사용할 때, 부모 클래스의 초기화 메서드를 호출합니다. 여기서는 파이토치의 nn.Module 클래스를 상속받는 MultivariateLinearRegression 클래스를 정의하고 있습니다. nn.Module 클래스는 파이토치 모델을 정의할 때 사용하는 기본 클래스이며, 초기화 메서드(init)를 포함하고 있습니다. 따라서 MultivariateLinearRegression 클래스에서는 nn.Module 클래스의 초기화 메서드를 호출해야 합니다. 그런데, 파이썬에서 다중 상속을 사용할 경우, 여러 개의 부모 클래스 중에서 어떤 클래스의 초기화 메서드를 호출해야 하는지가 모호해질 수 있습니다. 이를 해결하기 위해 파이썬에서는 super() 함수를 제공합니다. super() 함수는 현재 클래스(MultivariateLinearRegression)의 다음 부모 클래스(nn.Module)를 찾아서, 해당 클래스의 메서드(init)를 호출합니다. 이때, super() 함수에 첫 번째 인자로 현재 클래스(self)와 두 번째 인자로 self를 전달합니다. 이를 통해 현재 클래스(MultivariateLinearRegression)가 nn.Module 클래스를 상속받은 것임을 알리고, nn.Module 클래스의 초기화 메서드를 호출할 수 있게 됩니다. MultivariateLinearRegression 클래스의 다음 부모 클래스(nn.Module)의 초기화 메서드를 호출합니다.
    
    
* self.linear = nn.Linear(input_size, output_size)
    * self.linear은 인스턴스 변수(instance variable)입니다. 인스턴스 변수란, 클래스에서 생성된 객체(instance)의 속성(attribute)으로, 객체가 생성될 때마다 각각의 객체마다 고유한 값을 가지게 됩니다. self.linear은 MultivariateLinearRegression 클래스의 생성자(constructor)인 __init__ 메서드에서 초기화되었으며, 이후 클래스 내부의 다른 메서드에서도 사용될 수 있습니다. 따라서, self.linear는 MultivariateLinearRegression 클래스의 인스턴스(instance)의 속성(attribute)으로, 입력 특성과 출력 특성 사이의 선형 변환을 수행하는 신경망 계층(linear layer)을 나타냅니다.

In [3]:
# 모델 정의하기
class MultivariateLinearRegression(nn.Module):
    def __init__(self, input_size, output_size):
        super(MultivariateLinearRegression, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        out = self.linear(x)
        return out

# 모델 생성하기
model = MultivariateLinearRegression(X_train_tensor.shape[1], 1)

# 손실 함수와 최적화 알고리즘 정의하기
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

optimizer에 등록된 모든 매개변수의 gradient를 0으로 초기화하는 메서드입니다. 이를 호출하지 않으면, backward() 함수 호출 시 이전에 계산된 gradient 값과 현재 gradient 값이 누적되어 학습이 제대로 이루어지지 않을 수 있습니다.

따라서 모델의 학습을 시작하기 전에, optimizer.zero_grad()를 호출하여 gradient 값을 초기화해야 합니다. 예를 들어, 다음과 같은 코드에서는 각 학습 루프(iteration)마다 optimizer.zero_grad()를 호출하여 gradient를 초기화합니다.

In [10]:
# 모델 훈련하기
num_epochs = 5000
for epoch in range(num_epochs):
    # Forward 연산
    y_pred = model(X_train_tensor)
    
    # 손실 계산
    loss = criterion(y_pred.view(-1), y_train_tensor)
    
    # Backward 연산 및 가중치 갱신
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # 로그 정보 출력
    if (epoch+1) % 1000 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1000/5000], Loss: 11.5428
Epoch [2000/5000], Loss: 11.1792
Epoch [3000/5000], Loss: 11.1575
Epoch [4000/5000], Loss: 11.1391
Epoch [5000/5000], Loss: 11.1224


In [13]:
# 모델 평가하기
with torch.no_grad():
    y_pred_test = model(X_test_tensor)
    r2 = r2_score(y_test, y_pred_test.view(-1))
    print(f'R^2 Score: {r2:.4f}')

R^2 Score: 0.7887
